### Evaluation Class for Ablation Study

### Task/Dataset
- Task: RE
- Dataset: RE
### Required file
- `RE_test`: necessary file for ground truth, may contain multiple tags
- `all_relation`: all relation for RE

In [1]:
import pandas as pd
import numpy as np
import ast

In [5]:
time_dict = np.load('eval_result/time_dict_CTA_WebTable_7.npy',allow_pickle=True).item()
time_dict

{'init_select': 23.120656490325928,
 'init-train': 23.124510049819946,
 'batch-division': 46.78419876098633,
 'gradient-calculation': 522.2891850471497,
 'IF-Score': 570.9214282035828,
 'Final Selection': 571.1821782588959,
 'Selection fine-tune': 571.1865756511688,
 'Selection Inference': 571.1867270469666}

In [5]:
all_relation = np.load('train/RE/all_relation.npy',allow_pickle=True)
RE_test = pd.read_csv('train/RE/test_RAG.csv',index_col=0)


In [6]:
SimTab_test = pd.read_csv('inference_Transfer_ER/--RE-test.csv',index_col=0)
## Only for RE
SimTab_test['index'] = SimTab_test.index
def Ast(row):
    truth = list(eval(row['output']).values())[0]
    try:
        pred = list(eval(row['predict']).values())[0]
    except:
        pred = ''
    index = row['index']
    return truth, pred, index
SimTab_test_Transform = SimTab_test.apply(Ast,axis=1,result_type='expand')

In [7]:
SimTab_test_Transform.columns = ['truth','pred','index']
count = 0
truth_list = []
pred_list = []
result_metric = {}
for i in range(len(SimTab_test_Transform['index'].unique())):
    select_df = SimTab_test_Transform[SimTab_test_Transform['index'] == i]
    # truth = select_df.iloc[0,0]
    truth = eval(RE_test.iloc[i,1])
    select_df_filter = select_df[select_df['pred'].isin(all_relation)]
    try:
        pred = select_df_filter['pred'].value_counts().idxmax()
    except:
        pred = select_df['pred'].value_counts().idxmax()
    truth_list.append(truth)
    pred_list.append(pred)
    if truth.__contains__(pred):
        count += 1
    # else:
    #     print(truth,select_df['pred'].value_counts().idxmax())
result_metric['acc'] = count / len(SimTab_test_Transform['index'].unique())

In [8]:
result_metric

{'acc': 0.9005791505791506}

In [7]:
relation_dict = {}

for i in range(len(all_relation)):
    relation_dict[all_relation[i]] = i

In [15]:
from collections import defaultdict

def calculate_f1_scores(pred_list, truth_list):
    """
    根据给定的预测列表和真实值列表计算微平均F1 (micro-F1) 和宏平均F1 (macro-F1)。

    这个函数适用于每个预测对应一个真实值列表（其中包含一个或多个正确答案）的场景。

    Args:
        pred_list (list): 包含 n 个预测元素的列表。
        truth_list (list): 包含 n 个列表的列表，每个内层列表包含一个或多个正确的真实值。

    Returns:
        dict: 一个字典，包含 'micro_f1' 和 'macro_f1' 两个键值对。
              {'micro_f1': float, 'macro_f1': float}
    """
    # 步骤 1: 获取所有唯一的类别/标签
    all_labels = set(p for p in pred_list) | set(t for tl in truth_list for t in tl)

    # 步骤 2: 初始化 TP, FP, FN 计数器
    # 使用 defaultdict 可以让代码更简洁，当一个键第一次被访问时，会自动创建默认值（这里是0）。
    tp = defaultdict(int)
    fp = defaultdict(int)
    fn = defaultdict(int)

    # 步骤 3: 遍历每个预测和真实值对，填充 TP, FP, FN 计数器
    for pred, truths in zip(pred_list, truth_list):
        # 检查预测是否正确，来更新 TP 和 FP
        # pred: 当前的预测值
        # truths: 当前预测值对应的真实值列表
        if pred in truths:
            # 预测正确，该预测类别的 TP 加 1
            tp[pred] += 1
        else:
            # 预测错误，该预测类别的 FP 加 1
            fp[pred] += 1

        # 检查是否有漏掉的正确答案，来更新 FN
        for true_label in truths:
            # 如果真实值列表中的某个标签不等于我们的预测值，
            # 那么对于那个标签来说，这是一个 False Negative。
            if pred != true_label:
                fn[true_label] += 1

    # 步骤 4: 计算 Micro-F1 (微平均)
    # 首先，全局汇总所有类别的 TP, FP, FN
    total_tp = sum(tp.values())
    total_fp = sum(fp.values())
    total_fn = sum(fn.values())

    # 计算微平均 Precision 和 Recall
    micro_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    micro_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0

    # 计算微平均 F1
    micro_f1 = (2 * micro_precision * micro_recall) / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0

    # 步骤 5: 计算 Macro-F1 (宏平均)
    f1_scores_per_class = []
    # 遍历所有出现过的类别
    for label in all_labels:
        # 获取该类别的 TP, FP, FN
        label_tp = tp[label]
        label_fp = fp[label]
        label_fn = fn[label]

        # 计算该类别的 Precision 和 Recall
        precision = label_tp / (label_tp + label_fp) if (label_tp + label_fp) > 0 else 0
        recall = label_tp / (label_tp + label_fn) if (label_tp + label_fn) > 0 else 0

        # 计算该类别的 F1 分数
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores_per_class.append(f1)

    # 宏平均 F1 是所有类别 F1 分数的算术平均值
    macro_f1 = sum(f1_scores_per_class) / len(f1_scores_per_class) if f1_scores_per_class else 0

    return {
        'micro_f1': micro_f1,
        'macro_f1': macro_f1
    }
f1_metric = calculate_f1_scores(pred_list=pred_list,truth_list=truth_list)
for key in f1_metric:
    result_metric[key] = f1_metric[key]
print(result_metric)

{'micro_f1': 0.7723509933774834, 'macro_f1': 0.583034467461652}

## Evaluation for SimTab

In [1]:
from src.evaluation import evaluation
evaluator = evaluation()
import pandas as pd
# SimTab_test = pd.read_csv('inference_Transfer_ER/--SimTab_test_few.csv')
# all_metrics = evaluator.process('CTA','SimTab',SimTab_test)

Micro Score: 0.8462549277266754

Macro Score: 0.7313185684839995


In [2]:
result = pd.read_json('/data/home/wangys/geil/GEIL_Data/beers/correction/test.json')
len(result)

3364

In [6]:
import numpy as np
metrics = np.load('eval_result/mistral-7B-DC-beer.npy',allow_pickle=True).item()
# metrics
task = 'DC'
dataset = 'beer'
select_file = pd.read_json('train/{}/{}/train-select.json'.format(task,dataset))

In [7]:
len(select_file)

1305

In [7]:
import torch
task = 'ER'
dataset = 'semi-text-w'
sample_IF = torch.load('Influence/{}/{}/score.pkl'.format(task,dataset),weights_only=False)

In [10]:
batch_sampler = torch.load('Influence/{}/{}/batch.pkl'.format(task,dataset),weights_only=False)

In [16]:
output_file = pd.read_csv('output/Ablation/qwen2.5-7B-ER-semi-text-w-w-ppl.csv',index_col=0)
evaluator.process('ER','semi-text-w',output_file)

KeyError: 'prediction'

In [19]:
train_file = pd.read_json('train/ER/semi-text-w/semi-text-w-train.json')
len(train_file)

4258

In [21]:
batch_single = []
for i in  range(len(train_file)):
    batch_single.append([i])
# batch_single
task = 'ER'
dataset_name = 'semi-text-w'
batch_divide_path = 'Influence/{}/{}/batch_single.pkl'.format(task,dataset_name)
torch.save(batch_single,batch_divide_path)

In [23]:
sample_IF = torch.load('Influence_single/{}/{}/score.pkl'.format(task,dataset),weights_only=False)

In [44]:
result = torch.load('Influence_single/{}/{}/result.pkl'.format(task,dataset),weights_only=False)

In [47]:
result_df = pd.read_csv('Influence_single/ER/semi-text-w/result_df.csv',index_col=0)

In [56]:
def split_list(input_list, n):
    """
    将给定的列表划分为n个元素数量大体相当的子列表。

    参数:
    input_list (list): 需要划分的列表
    n (int): 要划分成的子列表个数

    返回:
    list: 划分后的n个子列表组成的列表
    """
    list_length = len(input_list)
    if n <= 0:
        raise ValueError("n必须大于0")
    if list_length == 0:
        return [[] for _ in range(n)]
    base_size = list_length // n
    remainder = list_length % n
    result = []
    start_index = 0
    for i in range(n):
        sublist_size = base_size + (1 if remainder > 0 else 0)
        sublist = input_list[start_index:start_index + sublist_size]
        result.append(sublist)
        start_index += sublist_size
        remainder -= 1
    return result

### 下面我们尝试依序查看之前的结果是否有错误，在tr_index上

In [68]:
task = 'ER'
dataset = 'semi-text-c'
import os
# tr_index = []
for i in range(1,9,1): ## maximum of k process
    if os.path.exists('grad/{}/{}/tr_grad_{}.pkl'.format(task,dataset,i)): ## i-th gradient 
        ## tr_grad
        # tr_grad_dict_part = torch.load('grad_single/{}/{}/tr_grad_{}.pkl'.format(task,dataset,i))
        # for key in tr_grad_dict_part.keys():
        
        ## tr_grad_index
        tr_index_part = torch.load('grad/{}/{}/tr_grad_index_{}.pkl'.format(task,dataset,i))
        # tr_index.extend(tr_index_part)
        print(len(tr_index_part))

92
92
92
91
91
91
91
91


In [81]:
if os.path.exists('Influence/{}/{}/score.pkl'.format(task,dataset)):
    print('exist')
    score = torch.load('Influence_single/{}/{}/score.pkl'.format(task,dataset),weights_only=False)

exist


In [83]:
score.keys()

dict_keys(['iterative', 'identity', 'proposed', 'LiSSA'])

In [89]:
list(score['LiSSA'].keys())[:10]

[3834, 2464, 670, 974, 1382, 3962, 1384, 2191, 973, 517]

In [90]:
list(score['proposed'].keys())[:10]

[1382, 1630, 1885, 3962, 2464, 1735, 327, 2534, 3546, 3834]

In [91]:
list(score['iterative'].keys())[:10]

[3834, 3625, 1296, 974, 3546, 2240, 1295, 1349, 973, 2464]

In [93]:
from src.util_func import get_top_k_indices

In [106]:
task = 'RE'
dataset = 'RE'

In [110]:
select_num = 1944
select_IF_single = {}
if os.path.exists('Influence_single/{}/{}'.format(task,dataset)):
    print('exits')
    sample_IF_single = torch.load('Influence_single/{}/{}/score.pkl'.format(task,dataset),weights_only=False)
    for IF_method in sample_IF_single.keys():
        select_IF_single[IF_method] = get_top_k_indices(sample_IF_single[IF_method],k=select_num,IF=True)

exits


In [3]:
metrics= np.load('eval_result/mistral-7B-RE-RE.npy',allow_pickle=True).item()
print(metrics)

{'IF-single-iterative': {'acc': 0.8711389961389961, 'micro_f1': 0.7471026490066226, 'macro_f1': 0.5905163780263318}, 'IF-single-identity': {'acc': 0.8426640926640927, 'micro_f1': 0.722682119205298, 'macro_f1': 0.55191678272745}, 'IF-single-proposed': {'acc': 0.8537644787644788, 'micro_f1': 0.732201986754967, 'macro_f1': 0.5632134148611553}, 'main': {'acc': 0.9005791505791506, 'micro_f1': 0.7723509933774834, 'macro_f1': 0.5830344674616523}, 'ppl': {'acc': 0.8513513513513513, 'micro_f1': 0.7301324503311257, 'macro_f1': 0.5310203623037854}, 'FL': {'acc': 0.9073359073359073, 'micro_f1': 0.7781456953642383, 'macro_f1': 0.5600671023778724}, 'IF': {'acc': 0.8388030888030888, 'micro_f1': 0.7193708609271523, 'macro_f1': 0.5556746386823592}, 'QuRating': {'acc': 0.915057915057915, 'micro_f1': 0.7847682119205298, 'macro_f1': 0.5921038653453791}}


In [4]:
metrics

{'IF-single-iterative': {'acc': 0.8711389961389961,
  'micro_f1': 0.7471026490066226,
  'macro_f1': 0.5905163780263318},
 'IF-single-identity': {'acc': 0.8426640926640927,
  'micro_f1': 0.722682119205298,
  'macro_f1': 0.55191678272745},
 'IF-single-proposed': {'acc': 0.8537644787644788,
  'micro_f1': 0.732201986754967,
  'macro_f1': 0.5632134148611553},
 'main': {'acc': 0.9005791505791506,
  'micro_f1': 0.7723509933774834,
  'macro_f1': 0.5830344674616523},
 'ppl': {'acc': 0.8513513513513513,
  'micro_f1': 0.7301324503311257,
  'macro_f1': 0.5310203623037854},
 'FL': {'acc': 0.9073359073359073,
  'micro_f1': 0.7781456953642383,
  'macro_f1': 0.5600671023778724},
 'IF': {'acc': 0.8388030888030888,
  'micro_f1': 0.7193708609271523,
  'macro_f1': 0.5556746386823592},
 'QuRating': {'acc': 0.915057915057915,
  'micro_f1': 0.7847682119205298,
  'macro_f1': 0.5921038653453791}}

In [1]:
def count_overlapping_elements(list1, list2):
    # This line (and the ones below it) should be indented 4 spaces
    set1 = set(list1)
    set2 = set(list2)

    # Using intersection to find common elements
    overlapping_elements = set1.intersection(set2)

    return len(overlapping_elements)

In [2]:
# import plotly.io as pio
# pio.renderers.default = 'iframe_connected'
import numpy as np
list1 = np.load('QuRating/data/select_index/RE-RE-QuRating.npy')
list2 = np.load('QuRating/data/select_index_main/RE-RE-QuRating.npy')
count_overlapping_elements(list1,list2)

451

In [116]:
count_overlapping_elements(select_IF_single['proposed'],select_IF_single['identity'])

1728

In [117]:
train = pd.read_json('train/DC/hospital/hospital-train.json')
len(train)

3744

### Checking Metrics

### CTA-WebTable

In [9]:
import numpy as np
import pandas as pd
from src.evaluation import evaluation
evaluator = evaluation()
task = 'RE'
dataset = 'RE'
# result = pd.read_csv('inference_Transfer_ER/--WebTable_Test_few.csv',index_col=0)
result = pd.read_csv('/data/home/wangys/DataSelection-IF/inference_Transfer_ER/--RE-test.csv',index_col=0)
metrics = evaluator.process(task,dataset,result)
all_metrics = np.load('eval_result/mistral-7B-{}-{}.npy'.format(task,dataset),allow_pickle=True).item()
all_metrics['main'] = metrics
# all_metrics

In [21]:
import numpy as np
import pandas as pd
from src.evaluation import evaluation
evaluator = evaluation()
task = 'ER'
dataset = 'semi-text-w'
all_metrics = {}
for ablation in ['ppl','FL','IF','main']:
    result = pd.read_csv('output/Ablation/qwen2.5-7B-{}-{}-w-{}.csv'.format(task,dataset,ablation),index_col=0)
    metrics = evaluator.process(task,dataset,result)
    all_metrics[ablation] = metrics
print(all_metrics)

Precision:0.5386996904024768

Recall:0.8246445497630331

F1:0.651685393258427
Precision:0.7427184466019418

Recall:0.7251184834123223

F1:0.7338129496402878
Precision:0.16965742251223492

Recall:0.985781990521327

F1:0.2894919972164231
Precision:0.7888888888888889

Recall:0.6729857819905213

F1:0.7263427109974424
{'ppl': {'precision': 0.5386996904024768, 'recall': 0.8246445497630331, 'F1': 0.651685393258427}, 'FL': {'precision': 0.7427184466019418, 'recall': 0.7251184834123223, 'F1': 0.7338129496402878}, 'IF': {'precision': 0.16965742251223492, 'recall': 0.985781990521327, 'F1': 0.2894919972164231}, 'main': {'precision': 0.7888888888888889, 'recall': 0.6729857819905213, 'F1': 0.7263427109974424}}


In [1]:
import numpy as np
import pandas as pd
from src.evaluation import evaluation
from src.util_func import load_yaml
evaluator = evaluation()
task = 'SM'
dataset = 'CMS'
config = load_yaml('script/config_{}_{}.yaml'.format(task,dataset))
select_config = load_yaml('script/ablation/mistral-7B_lora_{}_{}_P2-w-FL.yaml'.format(task,dataset))
train_file_path = config['train_file_path']
train_file = pd.read_json(train_file_path)
train_file_size = len(train_file)

select_file_path = select_config['train_file_path']
select_file = pd.read_json(select_file_path)
select_file_size = len(select_file)
print('Original Train File Size:{}\n\nSelect Train File Size:{}'.format(train_file_size,select_file_size))
all_metrics = {}

for ablation in ['ppl','FL','IF']:
    result = pd.read_csv('output/Ablation/{}/{}/mistral-7B-w-{}.csv'.format(task,dataset,ablation),index_col=0)
    
    metrics = evaluator.process(task,dataset,result)
    all_metrics[ablation] = metrics
# print(len(result))
print(all_metrics)

[2025-07-30 00:09:27,216] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/d

Original Train File Size:2563

Select Train File Size:522


/data/home/wangys/anaconda3/envs/verl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/home/wangys/anaconda3/envs/verl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision:0.0

Recall:0.0

F1:0.0


FileNotFoundError: [Errno 2] No such file or directory: 'output/Ablation/SM/CMS/mistral-7B-w-FL.csv'

In [7]:
dataset = 'CMS'
task = 'SM'
# result = pd.read_csv('inference_Transfer_ER/select--CMS-test.csv',index_col=0)
result = pd.read_csv('output/Ablation/SM/CMS/mistral-7B-w-FL.csv',index_col=0)
  
metrics = evaluator.process(task,dataset,result)

metrics

Precision:0.0

Recall:0.0

F1:0.0


/data/home/wangys/anaconda3/envs/verl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/home/wangys/anaconda3/envs/verl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'precision': 0.0, 'recall': 0.0, 'F1': 0.0}

In [19]:
# all_metrics
np.save('eval_result/mistral-7B-CTA-SimTab.npy',all_metrics)

In [8]:
import datasets

In [10]:
data = datasets.load_from_disk('QuRating/data/annotation/CTA-WebTable/')

In [29]:
data[0]['length']

672

In [5]:
670 * 1955

1309850

In [24]:
import glob
import datasets
subset = datasets.concatenate_datasets([datasets.load_from_disk(ds) for ds in sorted(glob.glob("QuRating/data/subset/RE-RE/*"))])


In [19]:
score_list = []
for i in range(len(subset)):
    score_list.extend([subset[i]['writing_style_average']])
    

In [34]:
np.histogram(np.array(list(score.values())))

(array([ 136, 2842, 3463,  202,    1,    0,    0,    7,    4,    2]),
 array([-1.828125  , -1.61046473, -1.39280446, -1.17514419, -0.95748392,
        -0.73982365, -0.52216339, -0.30450312, -0.08684285,  0.13081742,
         0.34847769]))

In [41]:
import torch
total_score = torch.load('selection/RE/RE/Total-Score.pkl',weights_only=False)
len(total_score)

6657

In [43]:
np.argsort(total_score)

array([ 765,  764, 1929, ..., 6461, 5476, 4390])

In [60]:
pd.read_json('train/RE/RE/train-select-w-QuRating.json')

,instruction,input,output
0,You are an expert in relation extraction from ...,,"{""relation"": ""aviation.airline.hubs""}"
1,You are an expert in relation extraction from ...,,"{""relation"": ""aviation.airline.hubs""}"
2,You are an expert in relation extraction from ...,,"{""relation"": ""aviation.airport.serves""}"
3,You are an expert in relation extraction from ...,,"{""relation"": ""aviation.airport.serves""}"
4,You are an expert in relation extraction from ...,,"{""relation"": ""aviation.airport.serves""}"
...,...,...,...
1287,You are an expert in relation extraction from ...,,"{""relation"": ""sports.sports_team.venue-sports...."
1288,You are an expert in relation extraction from ...,,"{""relation"": ""sports.sports_team.venue-sports...."
1289,You are an expert in relation extraction from ...,,"{""relation"": ""time.event.instance_of_recurring..."
1290,You are an expert in relation extraction from ...,,"{""relation"": ""tv.tv_program.original_network-t..."


In [68]:
cot = pd.read_json('../LESS/data/train/processed/flan_v2/flan_v2_data.jsonl',lines=True)

In [ ]:
import random
import json
import os
import yaml
from tqdm.notebook import tqdm
import pandas as pd
def process_datasets(config_folder: str, output_folder: str):
    """
    遍历给定文件夹内的 YAML 配置文件，处理相应的 JSON 文件，并生成 JSONL 格式的输出。

    Args:
        config_folder (str): 包含 YAML 配置文件的文件夹路径。
        output_folder (str): 用于保存新生成的 JSONL 文件的文件夹路径。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    for root, _, files in os.walk(config_folder):
        for file_name in files:
            if file_name.endswith('.yaml') and 'config_' in file_name:
                yaml_path = os.path.join(root, file_name)
                
                print(f"\nProcessing YAML file: {yaml_path}")
                try:
                    with open(yaml_path, 'r', encoding='utf-8') as f:
                        yaml_data = yaml.safe_load(f)
                except Exception as e:
                    print(f"Error reading YAML file {yaml_path}: {e}")
                    continue

                if 'train_file_path' in yaml_data:
                    json_path = yaml_data['train_file_path']
                    dataset_name = yaml_data.get('dataset', 'unknown_dataset')
                    task_name = yaml_data.get('task', 'unknown_task')
                    
                    full_dataset_name = f"{task_name}_{dataset_name}"
                    
                    if not os.path.exists(json_path):
                        print(f"Warning: JSON file not found at {json_path}. Skipping.")
                        continue
                        
                    print(f"  -> Reading JSON file: {json_path}")
                    try:
                        df = pd.read_json(json_path)
                    except Exception as e:
                        print(f"Error reading JSON file {json_path}: {e}")
                        continue
                    
                    # 检查所需的列是否存在
                    if 'instruction' not in df.columns or 'output' not in df.columns:
                        print(f"Warning: JSON file {json_path} is missing 'instruction' or 'output' column. Skipping.")
                        continue
                        
                    # 创建新的 JSONL 文件名
                    full_output_file = os.path.join(output_folder, f"{full_dataset_name}_full.jsonl")
                    subset_output_file = os.path.join(output_folder, f"{full_dataset_name}_subset.jsonl")
                    
                    # 生成完整数据集
                    full_data_list = []
                    for index, row in tqdm(df.iterrows(), total=len(df), desc="  -> Formatting data"):
                        new_entry = {
                            "dataset": full_dataset_name,
                            "id": f"{full_dataset_name}_{index}",
                            "messages": [
                                {
                                    "role": "user",
                                    "content": row['instruction']
                                },
                                {
                                    "role": "assistant",
                                    "content": row['output']
                                }
                            ]
                        }
                        full_data_list.append(new_entry)
                    
                    # 保存完整数据集
                    with open(full_output_file, 'w', encoding='utf-8') as f:
                        for item in full_data_list:
                            f.write(json.dumps(item, ensure_ascii=False) + '\n')
                    print(f"  -> Successfully created full JSONL file: {full_output_file}")
                    
                    # 随机抽取1%的数据样本
                    sample_size = max(1, int(len(full_data_list) * 0.01))
                    subset_data_list = random.sample(full_data_list, sample_size)
                    
                    # 保存子集
                    with open(subset_output_file, 'w', encoding='utf-8') as f:
                        for item in subset_data_list:
                            f.write(json.dumps(item, ensure_ascii=False) + '\n')
                    print(f"  -> Successfully created 1% subset JSONL file: {subset_output_file}")
def merge_jsonl_files(input_folder: str, output_folder: str):
    """
    将指定文件夹内的所有 full.jsonl 和 subset.jsonl 文件合并。

    Args:
        input_folder (str): 包含要合并的 JSONL 文件的文件夹路径。
        output_folder (str): 用于保存合并后的文件的文件夹路径。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    # 合并 full.jsonl 文件
    full_output_path = os.path.join(output_folder, 'full.jsonl')
    full_file_list = [f for f in os.listdir(input_folder) if f.endswith('_full.jsonl')]
    
    if not full_file_list:
        print("No files ending with '_full.jsonl' were found. Skipping full file merge.")
    else:
        print(f"Found {len(full_file_list)} full files to merge.")
        with open(full_output_path, 'w', encoding='utf-8') as outfile:
            for file_name in full_file_list:
                file_path = os.path.join(input_folder, file_name)
                print(f"  -> Merging: {file_name}")
                with open(file_path, 'r', encoding='utf-8') as infile:
                    for line in infile:
                        # 逐行读取并写入到新的文件中
                        outfile.write(line)
        print(f"Successfully merged all full files into: {full_output_path}")
        
    # ---

    # 合并 subset.jsonl 文件
    subset_output_path = os.path.join(output_folder, 'subset.jsonl')
    subset_file_list = [f for f in os.listdir(input_folder) if f.endswith('_subset.jsonl')]

    if not subset_file_list:
        print("No files ending with '_subset.jsonl' were found. Skipping subset file merge.")
    else:
        print(f"Found {len(subset_file_list)} subset files to merge.")
        with open(subset_output_path, 'w', encoding='utf-8') as outfile:
            for file_name in subset_file_list:
                file_path = os.path.join(input_folder, file_name)
                print(f"  -> Merging: {file_name}")
                with open(file_path, 'r', encoding='utf-8') as infile:
                    for line in infile:
                        # 逐行读取并写入到新的文件中
                        outfile.write(line)
        print(f"Successfully merged all subset files into: {subset_output_path}")


In [ ]:
process_datasets('script','LESS_data')

In [ ]:
merge_jsonl_files('LESS_data','LESS_data')

In [59]:
import pandas as pd
pd.read_json('/data/home/wangys/DataSelection-IF/train/AVE/oa_mine/train-select-w-main-QuRating.json')

FileNotFoundError: File /data/home/wangys/DataSelection-IF/train/AVE/oa_mine/train-select-w-main-QuRating.json does not exist

In [49]:
total_score[np.argsort(-total_score)[1]]

19.41932214796543

In [36]:
from src.util_func import get_top_k_indices
get_top_k_indices(score,k = 30)

[3449,
 4177,
 4188,
 3535,
 3574,
 3478,
 3557,
 4613,
 3836,
 3833,
 3821,
 3555,
 4140,
 3990,
 4583,
 4565,
 2933,
 3239,
 3642,
 3610,
 3779,
 4239,
 2415,
 3575,
 3597,
 3993,
 4510,
 43,
 2916,
 4043]

In [30]:
score = np.load('QuRating/data/select_index_main/RE-RE-QuRating-score.npy',allow_pickle=True).item()
score

{0: -1.25,
 1: -1.421875,
 2: -1.3203125,
 3: -1.328125,
 4: -1.4453125,
 5: -1.296875,
 6: -1.2734375,
 7: -1.4609375,
 8: -1.21875,
 9: -1.34375,
 10: -1.25,
 11: -1.328125,
 12: -1.28125,
 13: -1.3984375,
 14: -1.3203125,
 15: -1.2890625,
 16: -1.4453125,
 17: -1.2890625,
 18: -1.3359375,
 19: -1.4453125,
 20: -1.265625,
 21: -1.2734375,
 22: -1.3515625,
 23: -1.3359375,
 24: -1.2578125,
 25: -1.34375,
 26: -1.328125,
 27: -1.3359375,
 28: -1.3515625,
 29: -1.3125,
 30: -1.2734375,
 31: -1.28125,
 32: -1.3515625,
 33: -1.40625,
 34: -1.40625,
 35: -1.2890625,
 36: -1.390625,
 37: -1.28125,
 38: -1.28125,
 39: -1.4921875,
 40: -1.3515625,
 41: -1.296875,
 42: -1.2734375,
 43: -1.078125,
 44: -1.25,
 45: -1.3359375,
 46: -1.15625,
 47: -1.359375,
 48: -1.1875,
 49: -1.3203125,
 50: -1.171875,
 51: -1.28125,
 52: -1.2890625,
 53: -1.203125,
 54: -1.328125,
 55: -1.3359375,
 56: -1.1953125,
 57: -1.3203125,
 58: -1.1328125,
 59: -1.1640625,
 60: -1.203125,
 61: -1.3046875,
 62: -1.22656

In [25]:
score_list_extend = []
for i in range(len(subset)):
    score_list_extend.extend([subset[i]['writing_style_average']])
    

In [23]:
np.array(score_list).max(),np.array(score_list).min()

(0.23374520153550865, -0.703125)

In [26]:
np.array(score_list_extend).max(),np.array(score_list_extend).min()

(0.23374520153550865, -0.71875)

In [27]:
index_list = np.load('QuRating/data/select_index/DI-walmart-QuRating.npy')
len(index_list)

1949

In [24]:
original_file = pd.read_json('train/CTA/WebTable/WebTable-train.json')
original_file.iloc[193,0]

'You are an expert in relation extraction from wikipedia web table to knowledge graph. Given table title and column pair for Table 1, please choose the most proper type from the provided options. Return in json format.\n\nColumn: col_1\n\nOptions:["company", "industry", "name"]\n\nOutput Format Example:\n\n{"type": ""}\n\nTable 1:\n\n{\'col_1\': "AARON\'S INC…", \'symbol\': \'AAN\'}\n{\'col_1\': \'ITT EDUCATI…\', \'symbol\': \'ESI\'}\n{\'col_1\': \'MARCUS CORP…\', \'symbol\': \'MCS\'}\n{\'col_1\': \'LINN ENERGY…\', \'symbol\': \'LINE\'}\n{\'col_1\': \'GULFMARK OF…\', \'symbol\': \'GLF\'}\n\nReference tables:\n\n{\'Table\': "{\'col_1\': \'\', \'type\': \'Water/Psychic\'}\\n{\'col_1\': \'\', \'type\': \'Normal/Ground\'}\\n{\'col_1\': \'\', \'type\': \'Ghost/Normal\'}\\n{\'col_1\': \'\', \'type\': \'Ice/Flying\'}", \'Column\': \'col_1\', \'type\': \'name\'}\n\n{\'Table\': "{\'col_1\': \'MARCUS CORP…\', \'symbol\': \'MCS\'}\\n{\'col_1\': \'LINN ENERGY…\', \'symbol\': \'LINE\'}\\n{\'col_1\'

In [25]:
index_list = []
for i in range(len(subset)):
    index_list.append(subset[i]['index'])

In [34]:
# len(index_list),select_file_size
np.save('QuRating/data/select_index/CTA-WebTable-QuRating.npy',index_list)

In [17]:
CUDA_VISIBLE_DEVICES=6 python -m data_tools.select_subset data/annotation/CTA-WebTable/ data/subset/CTA-WebTable/ \
    --metric_field writing_style_average \
    --seq_len_field length \
    --tokens 5000000 \
    --temperature 2.0 \
    --normalize \
    --num_workers 8

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3697754659.py, line 1)